# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [24]:
# import libraries
import re
import numpy as np
import pandas as pd
import nltk
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC


In [2]:
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///Disaster_DB.db')
#print(engine.table_names())
df = pd.read_sql_table('disaster_db', engine)
#df.drop_na()
X =  df['message']
y = df.drop(['id', 'original', 'message', 'genre'], axis=1)
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
category_names = list(y.columns)
print(category_names)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# build pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))])  

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test)
for i, column in enumerate(category_names):
        print("Evaluation for column: ", column)
        print(classification_report(y_test[column], y_pred[:, i]))

Evaluation for column:  related
             precision    recall  f1-score   support

          0       0.63      0.36      0.45      1553
          1       0.82      0.93      0.88      5001

avg / total       0.78      0.80      0.78      6554

Evaluation for column:  request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5461
          1       0.83      0.40      0.54      1093

avg / total       0.88      0.89      0.87      6554

Evaluation for column:  offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6554

Evaluation for column:  aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3854
          1       0.75      0.51      0.61      2700

avg / total       0.73      0.73      0.72      6554

Evaluation

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
print(classification_report(y_test, y_pred, target_names=category_names))
#print(y_test.columns)

                        precision    recall  f1-score   support

               related       0.82      0.93      0.88      5001
               request       0.83      0.40      0.54      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.75      0.51      0.61      2700
          medical_help       0.57      0.08      0.14       532
      medical_products       0.62      0.10      0.17       345
     search_and_rescue       0.86      0.04      0.07       165
              security       1.00      0.01      0.02       127
              military       0.62      0.05      0.09       197
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.28      0.42       408
                  food       0.84      0.41      0.55       723
               shelter       0.80      0.22      0.35       590
              clothing       0.81      0.14      0.23        95
                 money       0.71      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        #'vect__max_df': (0.5, 1.0),
        #'tfidf__use_idf': [True, False],
        'clf__estimator__n_estimators': [50, 100],
        'clf__estimator__min_samples_split': [2, 5]
    }

cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs=-1, verbose=10)

In [14]:
model_imprv = cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, score=0.24183704607873055, total= 3.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, score=0.24610924626182484, total= 3.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  6.8min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, score=0.2438577750648558, total= 3.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 


...

[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 36.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=50, score=0.23775370059514725, total= 2.6min
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=100 


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 38.9min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=100, score=0.24336283185840707, total= 5.0min
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=100, score=0.24397314617027768, total= 5.0min
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=100, score=0.2452311918205402, total= 5.0min


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 55.7min finished


In [149]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:

print(model_imprv.best_params_)
print(cv.best_estimator_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}
Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))])


In [18]:
def evaluate_model(model, X_test, y_test, category_names):
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, target_names=category_names))
    #for i, column in enumerate(category_names):
    #    print("Evaluation for column: ", column)
    #    print(classification_report(y_test[column], y_pred[:, i]))


In [19]:
evaluate_model(model_imprv, X_test, y_test, category_names)

                        precision    recall  f1-score   support

               related       0.81      0.96      0.88      5001
               request       0.90      0.44      0.59      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.78      0.61      0.69      2700
          medical_help       0.66      0.05      0.09       532
      medical_products       0.79      0.08      0.14       345
     search_and_rescue       1.00      0.04      0.08       165
              security       0.33      0.01      0.02       127
              military       0.50      0.03      0.05       197
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.23      0.36       408
                  food       0.88      0.46      0.61       723
               shelter       0.86      0.22      0.35       590
              clothing       0.82      0.09      0.17        95
                 money       0.78      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [40]:
# Try LinearSVC model

pipeline_svc = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LinearSVC()))])  

In [41]:
pipeline_svc.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__C', 'clf__estimator__class_weight', 'clf__estimator__dual', 'clf__estimator__fit_intercept', 'clf__estimator__intercept_scaling', 'clf__estimator__loss', 'clf__estimator__max_iter', 'clf__estimator__multi_class', 'clf__estimator__penalty', 'clf__estimator__random_state', 'clf__estimator__tol', 'clf__estimator__verbose', 'clf__estimator', 'clf__n_jobs'])

In [49]:
parameters2 = {
        'vect__ngram_range': ((1,1), (1, 2)),
        #'vect__max_df': (0.5, 1.0),
        #'clf__estimator__kernel': ['poly'], 
        #'clf__estimator__degree': [1, 3],
        'clf__estimator__C':[1, 10, 100]
    
    }

cv_2 = GridSearchCV(pipeline_svc, param_grid = parameters2, n_jobs=-1, verbose=10)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [59]:
y_train.drop('child_alone', axis=1, inplace=True)
y_train.head()

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
11833,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3620,1,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
18855,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8375,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1322,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [50]:

model2 = cv_2.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf__estimator__C=1, vect__ngram_range=(1, 1) ...................
[CV]  clf__estimator__C=1, vect__ngram_range=(1, 1), score=0.28608483368935, total=  11.4s
[CV] clf__estimator__C=1, vect__ngram_range=(1, 1) ...................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   17.2s remaining:    0.0s


[CV]  clf__estimator__C=1, vect__ngram_range=(1, 1), score=0.28181263350625574, total=  11.4s
[CV] clf__estimator__C=1, vect__ngram_range=(1, 1) ...................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   34.5s remaining:    0.0s


[CV]  clf__estimator__C=1, vect__ngram_range=(1, 1), score=0.2865862963528155, total=  11.3s
[CV] clf__estimator__C=1, vect__ngram_range=(1, 2) ...................


...

[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  3.0min remaining:    0.0s


[CV]  clf__estimator__C=10, vect__ngram_range=(1, 1), score=0.22066229207996338, total=  20.4s
[CV] clf__estimator__C=10, vect__ngram_range=(1, 2) ..................


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  3.5min remaining:    0.0s


[CV]  clf__estimator__C=10, vect__ngram_range=(1, 2), score=0.2680805614891669, total=  46.2s
[CV] clf__estimator__C=10, vect__ngram_range=(1, 2) ..................
[CV]  clf__estimator__C=10, vect__ngram_range=(1, 2), score=0.26731766859932865, total=  48.0s
[CV] clf__estimator__C=10, vect__ngram_range=(1, 2) ..................
[CV]  clf__estimator__C=10, vect__ngram_range=(1, 2), score=0.2655272394323211, total=  49.2s
[CV] clf__estimator__C=100, vect__ngram_range=(1, 1) .................
[CV]  clf__estimator__C=100, vect__ngram_range=(1, 1), score=0.16890448581019224, total=  33.1s
[CV] clf__estimator__C=100, vect__ngram_range=(1, 1) .................
[CV]  clf__estimator__C=100, vect__ngram_range=(1, 1), score=0.15807140677448886, total=  33.9s
[CV] clf__estimator__C=100, vect__ngram_range=(1, 1) .................
[CV]  clf__estimator__C=100, vect__ngram_range=(1, 1), score=0.1655730199908439, total=  35.6s
[CV] clf__estimator__C=100, vect__ngram_range=(1, 2) .................
[CV]

[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 15.3min finished


In [57]:
print(model_imprv.best_params_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}


In [61]:
y_test.drop('child_alone', axis=1, inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [63]:
category_names.remove('child_alone')
category_names

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [64]:

evaluate_model(model2, X_test, y_test, category_names)

                        precision    recall  f1-score   support

               related       0.86      0.91      0.89      5001
               request       0.80      0.61      0.69      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.73      0.68      0.70      2700
          medical_help       0.59      0.30      0.40       532
      medical_products       0.66      0.29      0.41       345
     search_and_rescue       0.68      0.15      0.25       165
              security       0.57      0.03      0.06       127
              military       0.54      0.27      0.36       197
                 water       0.75      0.57      0.65       408
                  food       0.83      0.69      0.75       723
               shelter       0.76      0.56      0.64       590
              clothing       0.75      0.40      0.52        95
                 money       0.54      0.26      0.35       138
        missing_people       0.85      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [55]:
import pickle
# Pickle best model
pickle.dump(model_imprv, open('disaster_model.sav', 'wb'))

In [65]:
pickle.dump(model2, open('disaster_model_2.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.